In [2]:
from datasets import load_dataset

# Load a small slice first to test
ds = load_dataset("wikimedia/wikipedia", "20231101.en", split="train[:1%]")

print(ds[0])


s:\ML-project\GEN-AI-H2S-notebooks-\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'id': '12', 'url': 'https://en.wikipedia.org/wiki/Anarchism', 'title': 'Anarchism', 'text': 'Anarchism is a political philosophy and movement that is skeptical of all justifications for authority and seeks to abolish the institutions it claims maintain unnecessary coercion and hierarchy, typically including nation-states, and capitalism. Anarchism advocates for the replacement of the state with stateless societies and voluntary free associations. As a historically left-wing movement, this reading of anarchism is placed on the farthest left of the political spectrum, usually described as the libertarian wing of the socialist movement (libertarian socialism).\n\nHumans have lived in societies without formal hierarchies long before the establishment of states, realms, or empires. With the rise of organised hierarchical bodies, scepticism toward authority also rose. Although traces of anarchist ideas are found all throughout history, modern anarchism emerged from the Enlightenment. During

In [3]:
import json

fever_unified = []

with open("train.jsonl", "r") as f:
    for line in f:
        item = json.loads(line)
        if item['label'] != "SUPPORTS":
            continue  # skip REFUTES / NOT ENOUGH INFO

        for eg_idx, evidence_group in enumerate(item['evidence']):
            for ev_idx, ev in enumerate(evidence_group):
                article_title = ev[2] if ev[2] is not None else "NA"
                passage_id = f"fever_{item['id']}_{eg_idx}_{ev_idx}"
                entry = {
                    "id": passage_id,
                    "url": f"https://en.wikipedia.org/wiki/{article_title}" if article_title != "NA" else "NA",
                    "title": article_title,
                    "text": item['claim']  # simplest: use claim as text; later can replace with actual evidence text if available
                }
                fever_unified.append(entry)

print(f"Total SUPPORTS entries: {len(fever_unified)}")


FileNotFoundError: [Errno 2] No such file or directory: 'train.jsonl'

In [ ]:
fever_unified

[{'id': 'fever_75397_0_0',
  'url': 'https://en.wikipedia.org/wiki/Nikolaj_Coster-Waldau',
  'title': 'Nikolaj_Coster-Waldau',
  'text': 'Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.'},
 {'id': 'fever_75397_0_1',
  'url': 'https://en.wikipedia.org/wiki/Fox_Broadcasting_Company',
  'title': 'Fox_Broadcasting_Company',
  'text': 'Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.'},
 {'id': 'fever_150448_0_0',
  'url': 'https://en.wikipedia.org/wiki/Roman_Atwood',
  'title': 'Roman_Atwood',
  'text': 'Roman Atwood is a content creator.'},
 {'id': 'fever_150448_1_0',
  'url': 'https://en.wikipedia.org/wiki/Roman_Atwood',
  'title': 'Roman_Atwood',
  'text': 'Roman Atwood is a content creator.'},
 {'id': 'fever_214861_0_0',
  'url': 'https://en.wikipedia.org/wiki/History_of_art',
  'title': 'History_of_art',
  'text': 'History of art includes architecture, dance, sculpture, music, painting, poetry literature, theatre, narrative, film, photography and graphic

In [ ]:
import pandas as pd
fev_ds = pd.DataFrame(fever_unified)

In [ ]:
ds_wiki = ds.to_pandas()

In [ ]:
ds_wiki.head()

,id,url,title,text
0,12,https://en.wikipedia.org/wiki/Anarchism,Anarchism,Anarchism is a political philosophy and moveme...
1,39,https://en.wikipedia.org/wiki/Albedo,Albedo,Albedo (; ) is the fraction of sunlight that i...
2,290,https://en.wikipedia.org/wiki/A,A,"A, or a, is the first letter and the first vow..."
3,303,https://en.wikipedia.org/wiki/Alabama,Alabama,Alabama () is a state in the Southeastern regi...
4,305,https://en.wikipedia.org/wiki/Achilles,Achilles,"In Greek mythology, Achilles ( ) or Achilleus ..."


In [ ]:
df_ret = pd.concat([ds_wiki, fev_ds])

In [ ]:
df_ret

,id,url,title,text
0,12,https://en.wikipedia.org/wiki/Anarchism,Anarchism,Anarchism is a political philosophy and moveme...
1,39,https://en.wikipedia.org/wiki/Albedo,Albedo,Albedo (; ) is the fraction of sunlight that i...
2,290,https://en.wikipedia.org/wiki/A,A,"A, or a, is the first letter and the first vow..."
3,303,https://en.wikipedia.org/wiki/Alabama,Alabama,Alabama () is a state in the Southeastern regi...
4,305,https://en.wikipedia.org/wiki/Achilles,Achilles,"In Greek mythology, Achilles ( ) or Achilleus ..."
...,...,...,...,...
193751,fever_13114_0_0,https://en.wikipedia.org/wiki/Gimli_-LRB-Middl...,Gimli_-LRB-Middle-earth-RRB-,J. R. R. Tolkien created Gimli.
193752,fever_13114_1_0,https://en.wikipedia.org/wiki/Gimli_-LRB-Middl...,Gimli_-LRB-Middle-earth-RRB-,J. R. R. Tolkien created Gimli.
193753,fever_152180_0_0,https://en.wikipedia.org/wiki/Susan_Sarandon,Susan_Sarandon,Susan Sarandon is an award winner.
193754,fever_152180_1_0,https://en.wikipedia.org/wiki/Susan_Sarandon,Susan_Sarandon,Susan Sarandon is an award winner.


In [ ]:
df_ret_texts_list = df_ret['text'].to_list()

In [ ]:
type(df_ret_texts_list)

NameError: name 'df_ret_texts_list' is not defined

In [ ]:
from dotenv import load_dotenv
import os
import requests

load_dotenv()
API_key = os.getenv("GOOGLE_FACT_CHECK_API")


url = "https://factchecktools.googleapis.com/v1alpha1/claims:search"

params = {
    'key': API_key,
    'query': 'covid vaccine', 
    'languageCode': 'en-US',
    'pageSize': 100
}

response = requests.get(url,params=params)
len(response.json()['claims'])

100

In [52]:
response.json()['claims'][0]
i=0
for claim in response.json()['claims']:
    # print(claim.get('claimReview')[0].get('title',''))
    print(claim.get('claimReview')[0].get('reviewDate',''))
    i+=1
    if i==4:
        break



2025-09-08T00:00:00Z
2025-09-08T00:00:00Z
2025-09-08T00:00:00Z
2025-09-04T20:58:00Z


In [ ]:
def fetch_google_facts(query,num_iter = 1, pages = 100):
    
    load_dotenv()
    API_key = os.getenv("GOOGLE_FACT_CHECK_API")


    url = "https://factchecktools.googleapis.com/v1alpha1/claims:search"

    ds=[]
    next_page_token = None
    for _ in range(num_iter):

        params = {
        'key': API_key,
        'query': query, 
        'languageCode': 'en-US',
        'pageSize': pages
        }

        if next_page_token:
            params['pageToken'] = next_page_token

        try:
            response = requests.get(url,params=params)
            response.raise_for_status()

            for claim in response.json()['claims']:
                ds.append(
                    {'title':claim.get('claimReview',[])[0].get('title',''),
                     'text':claim.get('text',''),
                     'url':claim.get('claimReview',[])[0].get('url',''), 
                     'reviewDate':claim.get('claimReview')[0].get('reviewDate','')
                     })
                
            next_page_token = response.json().get('nextPageToken')
            if not next_page_token:
                break

        except requests.exceptions.RequestException as e:
            print(f"Request error: {e}")
            break
        except Exception as e:
            print(f"Error processing response: {e}")
            break

    return ds


In [61]:
ds1 = fetch_google_facts('indian government')
ds1,len(ds1)

([{'title': 'Did Govt Declare Sept 3 to 6, 2025, as Public Holidays? Fact Check – DigitEye India',
   'text': 'The Indian Government has declared September 3-6 2025 as public holidays.',
   'url': 'https://digiteye.in/did-govt-declare-sept-3-to-6-2025-as-public-holidays-fact-check/',
   'reviewDate': '2025-09-05T00:00:00Z'},
  {'title': 'Viral ₹1 Lakh Rupee Coin Is Not Issued by the Government of India',
   'text': 'Viral video shows a genuine one lakh rupee coin issued by the Indian government.',
   'url': 'https://factly.in/viral-%E2%82%B91-lakh-rupee-coin-is-not-issued-by-the-government-of-india/',
   'reviewDate': '2025-09-05T00:00:00Z'},
  {'title': 'Fact Check: Has Indian Govt Promised To Bring Back A Truck Driver ...',
   'text': 'Indian Govt Promised To Bring Back Harjinder Singh From US In 24 Hours',
   'url': 'https://www.newsmobile.in/nm-fact-checker/fact-check-indian-govt-has-not-promised-to-bring-back-harjinder-singh-from-us-in-24-hours/',
   'reviewDate': '2025-09-03T00:0

In [60]:
ds2 = fetch_google_facts('indian politics')
len(ds2)

9

In [ ]:
# ds = ds1 + ds2
len(ds)

209

In [ ]:
import pandas as pd
df = pd.DataFrame(ds)


,title,text,url,reviewDate
0,"Did Govt Declare Sept 3 to 6, 2025, as Public ...",The Indian Government has declared September 3...,https://digiteye.in/did-govt-declare-sept-3-to...,2025-09-05T00:00:00Z
1,Viral ₹1 Lakh Rupee Coin Is Not Issued by the ...,Viral video shows a genuine one lakh rupee coi...,https://factly.in/viral-%E2%82%B91-lakh-rupee-...,2025-09-05T00:00:00Z
2,Fact Check: Has Indian Govt Promised To Bring ...,Indian Govt Promised To Bring Back Harjinder S...,https://www.newsmobile.in/nm-fact-checker/fact...,2025-09-03T00:00:00Z
3,"The Indian government has not declared 3, 4, 5...","The Indian government has declared 3, 4, 5, an...",https://factly.in/the-indian-government-has-no...,2025-09-01T00:00:00Z
4,Hotmail founder Sabeer Bhatia claims Govt is s...,While the cost of a Vande Bharat train is Rs 1...,https://digiteye.in/hotmail-founder-sabeer-bha...,2025-08-20T00:00:00Z
...,...,...,...,...
204,Viral audio clip with commentary on Indian pol...,"Speech given by Lakshmi Mittal on PM Modi, Ind...",https://www.altnews.in/fake-audio-claiming-to-...,2019-09-23T00:00:00Z
205,Fact Check: Did Lakshmi Mittal Comment on Esta...,Lakshmi Mittal comments on religion and politi...,https://www.newsmobile.in/nm-fact-checker/fact...,2023-01-19T00:00:00Z
206,Video of Indian YouTuber falsely shared as 've...,IPS officer speaking against corruption in Ind...,https://factcheck.afp.com/doc.afp.com.32HD74M,2022-09-02T09:03:00Z
207,Man Talking About Politics in Viral Video Is N...,The person speaking about the criminalisation ...,https://www.thequint.com/news/webqoof/misleadi...,2022-08-29T08:31:54Z
